In [ ]:
#chrome.exe -remote-debugging-port=9222 -user-data-dir=c:\chromeData --no-first-run
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import json
from datetime import datetime
DOWNLOADS_DIRECTORY = os.path.join(os.getcwd(),"downloads")
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#prefs = {"download.default_directory": DOWNLOADS_DIRECTORY,
#         "plugins.always_open_pdf_externally": True}
chrome_options.debugger_address="127.0.0.1:9222"
#chrome_options.add_experimental_option("prefs",prefs)
#chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=chrome_options)
url = driver.command_executor._url
driver.session_id

driver.get('https://eu-webapp.spotme.com/api/v1/content/0ac3366f4136af65271b380d428ffd1d/contents?skip=0&fp_type=content-document&limit=2000')

itms = json.loads(driver.find_element(By.TAG_NAME,"pre").text)
print(json.dumps(itms["items"][0:3],indent=2))

import pandas as pd
df = pd.DataFrame(itms["items"])
df['tags'].apply(lambda x: ';'.join(x))
df.describe()

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from dataTable import DataTable
from storage import check_blob_exists,create_blob
METADATA_TABLE_NAME = "CyberMetadata"
OUTPUT_FOLDER_NAME_PDF = "public-pdf/"
OUTPUT_CONTAINER_NAME = "ai-workbench-cyber"
dataTab = DataTable(table_name=METADATA_TABLE_NAME,partition_key=OUTPUT_FOLDER_NAME_PDF.replace('/',''))



In [ ]:
driver.implicitly_wait(1)
for i, pdf_json in enumerate(itms["items"]): 
    file_id = pdf_json.get("id")
    dl_template = f"https://eu-webapp.spotme.com/api/v1/eid/0ac3366f4136af65271b380d428ffd1d/webapp/doc?id={file_id}&attachment=file"
    filename = file_id+".pdf"
    # if dataTab.check_url_exists(dl_template,None):
    #     continue
    if check_blob_exists(OUTPUT_CONTAINER_NAME,OUTPUT_FOLDER_NAME_PDF+filename):
        continue
    print(dl_template)
    #driver.get(dl_template)
    tags = ";".join(pdf_json["tags"])
    timestamp = datetime.utcfromtimestamp(pdf_json['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
    title = pdf_json["title"]
    metadata = f"{title} ; {tags} ; date:{timestamp} "
    dict_values = {
        "RowKey" : filename,
        "Source" : dl_template,
        "Metadata": metadata 
    }
    dataTab.create_entity_raw(dict_value=dict_values)
    print(i)

In [ ]:
PDF_METADATA_FILE_NAME="pdfMetadata.csv"
get_metadata = DataTable(
        table_name=METADATA_TABLE_NAME,
        partition_key=f"{OUTPUT_FOLDER_NAME_PDF.replace('/','')}",
    )
metadata_entities = get_metadata.query_entity_by_partition()
csv_data='\n'.join(list(map(lambda ls: ','.join(ls),metadata_entities)))
csv_data='\n'.join(list(map(lambda ls: ','.join(['"{}"'.format(val) for val in ls]),metadata_entities)))
create_blob(OUTPUT_CONTAINER_NAME,OUTPUT_FOLDER_NAME_PDF+PDF_METADATA_FILE_NAME,csv_data)